In [16]:
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pdc

In [37]:
train = pd.DataFrame({
    'device_id': [1, 2, 3, 4, 5],
    'brand': ['Iphone', 'Iphone', 'Nokia', 'Nokia',  'Samsung']
})
train

,device_id,brand
0,1,Iphone
1,2,Iphone
2,3,Nokia
3,4,Nokia
4,5,Samsung


In [38]:
# Generate col_ind and row_ind
encoder = LabelEncoder().fit(train['brand'])
col_ind = encoder.transform(train['brand'])

M = 5
N = 3
row_ind = np.arange(M)
data = np.ones(M)
train['trainrow'] = row_ind

# Generate row_ind
print("col index", col_ind)
print("row_index", row_ind)
print("data", data)


col index [0 0 1 1 2]
row_index [0 1 2 3 4]
data [1. 1. 1. 1. 1.]


In [39]:
new_data = csr_matrix((data, [row_ind, col_ind]), shape=(M, N))
new_data.todense()


matrix([[1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

## Grouped 1HE


In [109]:
apps = pd.DataFrame({
    'trainrow': [0, 0, 0, 1, 1],
    'device_id': [1, 1, 1, 2, 2],
    'event_id': [5, 5, 4, 3, 3],
    'app': ['EDX', 'Runtastic', 'Runtastic', 'EDX',  'WeatherApp']
})
apps

,trainrow,device_id,event_id,app
0,0,1,5,EDX
1,0,1,5,Runtastic
2,0,1,4,Runtastic
3,1,2,3,EDX
4,1,2,3,WeatherApp


In [110]:

appencoder = LabelEncoder().fit(apps['app'])
apps['app'] = appencoder.transform(apps['app'])

aggtable = (
    apps.groupby(['device_id', 'app'])['device_id']
    .agg(['count'])
    .reset_index()
    .merge(train[['device_id', 'trainrow']], how='left', on='device_id')
    .reset_index()
)

N = 3
xtr = csr_matrix(
    (aggtable['count'].values.astype(float), (aggtable.trainrow, aggtable['app'].values)),
    shape=(M, N))
    
print(aggtable)

   index  device_id  app  count  trainrow
0      0          1    0      1         0
1      1          1    1      2         0
2      2          2    0      1         1
3      3          2    2      1         1


In [111]:
xtr.todense()

matrix([[1., 2., 0.],
        [1., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])